In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.nn.utils.rnn import pack_padded_sequence
from torch.autograd import Variable
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
import numpy as np

In [ ]:
import json

In [ ]:
test_path = 'drive/MyDrive/DeepLearningProject_Group3/preprocessed/data_preprocessed/test.json'
dev_path = 'drive/MyDrive/DeepLearningProject_Group3/preprocessed/data_preprocessed/dev.json'

In [ ]:
with open(test_path, 'r') as file:
    test = json.load(file)

In [ ]:
x_test = test["one_hot"]
x_test_token = test["word_embedding"]
y_test = test["sentiments"]
y_test = np.array(y_test)

In [ ]:
with open(dev_path, 'r') as file:
    dev = json.load(file)

In [ ]:
x_dev = dev["one_hot"]
x_dev_token = dev["word_embedding"]
y_dev = dev["sentiments"]
y_dev = np.array(y_dev)

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# LSTM model
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size, max_sequence_length, bidirectional, sum_output):
        super().__init__()
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        self.max_sequence_length = max_sequence_length
        self.bidirectional = bidirectional
        self.sum_output = sum_output

        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

        if bidirectional:
            self.lstm = self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers, batch_first=True, bidirectional=True)
            self.fc = nn.Linear(2 * hidden_size, output_size)

    def forward(self, x):
        x = pad_sequences(x, maxlen=self.max_sequence_length, padding='post', truncating='post')
        x = torch.Tensor(x).to(device)

        h0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size)).to(device)
        c0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size)).to(device)
        if self.bidirectional:
            h0 = Variable(torch.zeros(2 * self.num_layers, x.size(0), self.hidden_size)).to(device)
            c0 = Variable(torch.zeros(2 * self.num_layers, x.size(0), self.hidden_size)).to(device)
        
        out, (h, c) = self.lstm(x, (h0, c0))

        if self.sum_output:
            out = torch.sum(out, axis=1) # Sum of all lstm cell output to feed into linear layer
        else:
            out = out[:, -1, :] # get the output at final time step to feed into linear layer

        out = self.fc(out)
        return out


class LSTMEmbeddedModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size, embedding_dim, max_sequence_length, bidirectional):
        super().__init__()
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        self.max_sequence_length = max_sequence_length
        self.bidirectional = bidirectional

        self.embedding = nn.Embedding(num_embeddings=input_size, embedding_dim=embedding_dim)
        self.lstm = nn.LSTM(input_size=embedding_dim, hidden_size=hidden_size, num_layers=num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

        if bidirectional:
            self.lstm = self.lstm = nn.LSTM(input_size=embedding_dim, hidden_size=hidden_size, num_layers=num_layers, batch_first=True, bidirectional=True)
            self.fc = nn.Linear(2 * hidden_size, output_size)

    def forward(self, x):
        x = pad_sequences(x, maxlen=self.max_sequence_length, padding='post', truncating='post')
        x = torch.LongTensor(x).to(device)
        x = self.embedding(x)

        h0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size)).to(device)
        c0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size)).to(device)
        if self.bidirectional:
            h0 = Variable(torch.zeros(2 * self.num_layers, x.size(0), self.hidden_size)).to(device)
            c0 = Variable(torch.zeros(2 * self.num_layers, x.size(0), self.hidden_size)).to(device)
        
        out, (h, c) = self.lstm(x, (h0, c0))

        out = torch.sum(out, axis=1) # get the output at final time step to feed into linear layer
        out = self.fc(out)

        return out

In [ ]:
model1 = torch.load("drive/MyDrive/DeepLearningProject_Group3/checkpoint/LSTM/token_model.pth")

In [ ]:
y_test_pred = model1(x_test_token)
softmax = nn.Softmax(dim=1)
y_test_pred_final = softmax(y_test_pred)
y_test_pred_final = y_test_pred_final.cpu().detach().numpy()

In [ ]:
y_dev_pred = model1(x_dev_token)
softmax = nn.Softmax(dim=1)
y_dev_pred_final = softmax(y_dev_pred)
y_dev_pred_final = y_dev_pred_final.cpu().detach().numpy()

In [ ]:
y_test_pred_final.shape

(3166, 3)

In [ ]:
y_dev_pred_final.shape

(1583, 3)

In [ ]:
import pickle

In [ ]:
save_path = "drive/MyDrive/DeepLearningProject_Group3/output_for_ensemble/lstm_output_test.pkl"
with open(save_path, "wb") as file:
    pickle.dump(y_test_pred_final, file)

In [ ]:
save_path = "drive/MyDrive/DeepLearningProject_Group3/output_for_ensemble/lstm_output_dev.pkl"
with open(save_path, "wb") as file:
    pickle.dump(y_dev_pred_final, file)

# Word Segmentation

In [ ]:
test_segment_path = 'drive/MyDrive/DeepLearningProject_Group3/preprocessed/word_segmentation/test.json'
dev_segment_path = 'drive/MyDrive/DeepLearningProject_Group3/preprocessed/word_segmentation/dev.json'

In [ ]:
with open(test_segment_path, 'r') as file:
    test_segment = json.load(file)

In [ ]:
x_test_segment = test_segment["one_hot"]
x_test_token_segment = test_segment["tokenizer_encoding"]
y_test_segment = test_segment["sentiments"]
y_test_segment = np.array(y_test_segment)

In [ ]:
with open(dev_segment_path, 'r') as file:
    dev_segment = json.load(file)

In [ ]:
x_dev_segment = dev_segment["one_hot"]
x_dev_token_segment = dev_segment["tokenizer_encoding"]
y_dev_segment = dev_segment["sentiments"]
y_dev_segment = np.array(y_dev_segment)

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
def evaluate(model, x_dev, y_dev):
    with torch.no_grad():
        output = model(x_dev)
        _, y_dev_pred = torch.max(output, 1)
        y_dev_pred = y_dev_pred.cpu().detach().numpy()
        if type(y_dev) == torch.Tensor:
            y_dev = y_dev.cpu().detach().numpy()
        print(classification_report(y_dev, y_dev_pred, digits=4))

In [ ]:
model2 = torch.load("drive/MyDrive/DeepLearningProject_Group3/checkpoint/LSTM/token_model_segment.pth")

In [ ]:
evaluate(model2, x_test_token_segment, y_test_segment)

              precision    recall  f1-score   support

           0     0.8923    0.9347    0.9130      1409
           1     0.4672    0.3832    0.4211       167
           2     0.9253    0.9038    0.9144      1590

    accuracy                         0.8901      3166
   macro avg     0.7616    0.7406    0.7495      3166
weighted avg     0.8864    0.8901    0.8878      3166



In [ ]:
y_test_pred_segment = model2(x_test_token_segment)
softmax = nn.Softmax(dim=1)
y_test_pred_final_segment = softmax(y_test_pred_segment)
y_test_pred_final_segment = y_test_pred_final_segment.cpu().detach().numpy()

In [ ]:
y_dev_pred_segment = model2(x_dev_token_segment)
softmax = nn.Softmax(dim=1)
y_dev_pred_final_segment = softmax(y_dev_pred_segment)
y_dev_pred_final_segment = y_dev_pred_final_segment.cpu().detach().numpy()

In [ ]:
import pickle

In [ ]:
save_path = "drive/MyDrive/DeepLearningProject_Group3/output_for_ensemble/lstm_output_test_segment.pkl"
with open(save_path, "wb") as file:
    pickle.dump(y_test_pred_final_segment, file)

In [ ]:
save_path = "drive/MyDrive/DeepLearningProject_Group3/output_for_ensemble/lstm_output_dev_segment.pkl"
with open(save_path, "wb") as file:
    pickle.dump(y_dev_pred_final_segment, file)

In [ ]:
y_dev_pred_final_segment.shape

(1583, 3)

In [ ]:
y_test_pred_final_segment.shape

(3166, 3)